In [24]:
# Notwendige Bibliotheken importieren
import pandas as pd
import requests
import random
import datetime

## Laden des Datensatzes

Jetzt laden wir unseren Datensatz. Wir gehen davon aus, dass der Datensatz eine CSV-Datei mit Verkaufsdaten enthält.

Fetch Public and public Parking Data from the APIs

In [25]:
# API URL for Freiburg
api_url_freiburg = "https://api.parkendd.de/Freiburg"

# Fetch the data from the WFS API
response_freiburg = requests.get(api_url_freiburg)

if response_freiburg.status_code == 200:
    public_parking_data = response_freiburg.json()

    # Extract relevant data
    parking_data_public = []
    for site in public_parking_data['lots']:

        if site.get("purpose") and site.get("purpose") != "CAR":
            continue
        
        coords = site.get("coords", {})
        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": site.get("address"),
            "latitude": coords.get("lat"),
            "longitude": coords.get("lng"),
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("total"),
            "available_space": site.get("free"),
            "opening_time": random.choice(["Weekdays 9AM-11PM", "Weekdays 9AM-7PM", "Weekdays 6AM-6PM", "Monday-Sunday 9AM-11PM"])
        })
    
    # Convert to DataFrame
    df_freiburg = pd.DataFrame(parking_data_public)
    print("Public Parking Data:", df_freiburg.head())
else:
    print("Failed to fetch MobiData BW parking data.")

Public Parking Data:                    public_id              name address   latitude  longitude  \
0        freiburgunterlinden       Unterlinden      P8  47.997868   7.848868   
1        freiburglandratsamt       Landratsamt     P14  48.000059   7.857081   
2         freiburgmartinstor        Martinstor     P18  47.992506   7.847365   
3        freiburgkonzerthaus       Konzerthaus      P2  47.995290   7.840836   
4  freiburgkollegiengebaeude  Kollegiengebäude     P19  47.993431   7.847403   

   price_per_hour  capacity  available_space            opening_time  
0             3.2       218              139        Weekdays 9AM-7PM  
1             3.8       202               14       Weekdays 9AM-11PM  
2             3.8       132                6  Monday-Sunday 9AM-11PM  
3             1.6       421              192        Weekdays 6AM-6PM  
4             1.6       129                1        Weekdays 9AM-7PM  


In [26]:
# URL for MobiData BW API (public parking data)
api_url_gebündelte_parkplätze_parkbauten = "https://api.mobidata-bw.de/park-api/api/public/v3/parking-sites"

# Fetch data from MobiData BW API
response_gebündelte_parkplätze_parkbauten = requests.get(api_url_gebündelte_parkplätze_parkbauten)

# Check the response and parse the JSON data
if response_gebündelte_parkplätze_parkbauten.status_code == 200:
    public_parking_data = response_gebündelte_parkplätze_parkbauten.json()
    
    # Extract relevant data (you can modify based on the actual response structure)
    parking_data_public = []
    for site in public_parking_data['items']:
        
        if site.get("purpose") and site.get("purpose") != "CAR":
            continue

        latitude = float(site.get("lat", 0)) 
        longitude = float(site.get("lon", 0))  
        
        if not (47.52 <= latitude <= 49.79 and 7.50 <= longitude <= 10.49):
            continue
        
        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": site.get("address"),
            "latitude": latitude,
            "longitude": longitude,
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("capacity"),
            "available_space": site.get("realtime_free_capacity"),
            "opening_time": random.choice(["Weekdays 9AM-11PM", "Weekdays 9AM-7PM", "Weekdays 6AM-6PM", "Monday-Sunday 9AM-11PM"])
        })
    
    # Convert to DataFrame
    df_public_parking = pd.DataFrame(parking_data_public)
    print("Public Parking Data:", df_public_parking.head())
else:
    print("Failed to fetch MobiData BW parking data.")


Public Parking Data:    public_id                         name  \
0        887  Listplatz 1 / Bahnhofstraße   
1        888          Obere Wässere 3 - 7   
2      17151                    Parkplatz   
3        757                Bahnhofstraße   
4        758                Am Südbahnhof   

                                   address   latitude  longitude  \
0  Listplatz 1 / Bahnhofstraße, Reutlingen  48.495983   9.210331   
1          Obere Wässere 3 - 7, Reutlingen  48.488955   9.216701   
2                                Stuttgart  48.834065   9.152340   
3                Bahnhofstraße, Reutlingen  48.497575   9.211077   
4                Am Südbahnhof, Reutlingen  48.482762   9.229821   

   price_per_hour  capacity  available_space       opening_time  
0             3.8     198.0              NaN  Weekdays 9AM-11PM  
1             1.6     181.0              NaN   Weekdays 9AM-7PM  
2             3.8     200.0              NaN  Weekdays 9AM-11PM  
3             1.6     128.0        

In [27]:
df_freiburg

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,freiburgunterlinden,Unterlinden,P8,47.997868,7.848868,3.2,218,139,Weekdays 9AM-7PM
1,freiburglandratsamt,Landratsamt,P14,48.000059,7.857081,3.8,202,14,Weekdays 9AM-11PM
2,freiburgmartinstor,Martinstor,P18,47.992506,7.847365,3.8,132,6,Monday-Sunday 9AM-11PM
3,freiburgkonzerthaus,Konzerthaus,P2,47.995290,7.840836,1.6,421,192,Weekdays 6AM-6PM
4,freiburgkollegiengebaeude,Kollegiengebäude,P19,47.993431,7.847403,1.6,129,1,Weekdays 9AM-7PM
5,freiburgggrafhalle,G.-Graf-Halle,P6,47.999300,7.846800,3.2,56,23,Monday-Sunday 9AM-11PM
6,freiburgkarlsbau,Karlsbau,P13,47.997446,7.853848,3.2,656,37,Weekdays 6AM-6PM
7,freiburgwestarkaden,Westarkaden,P22,48.006420,7.827394,3.2,750,333,Weekdays 9AM-11PM
8,freiburgschwabentor,Schwabentor,P16,47.991264,7.856310,3.8,221,113,Weekdays 6AM-6PM
9,freiburgzaehringertor,Zähringer Tor,P12,47.998958,7.854022,3.8,162,63,Weekdays 9AM-11PM


In [28]:
df_public_parking

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",48.495983,9.210331,3.8,198.0,NaN,Weekdays 9AM-11PM
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",48.488955,9.216701,1.6,181.0,NaN,Weekdays 9AM-7PM
2,17151,Parkplatz,Stuttgart,48.834065,9.152340,3.8,200.0,NaN,Weekdays 9AM-11PM
3,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",48.497575,9.211077,1.6,128.0,NaN,Weekdays 9AM-7PM
4,758,Am Südbahnhof,"Am Südbahnhof, Reutlingen",48.482762,9.229821,3.8,NaN,NaN,Weekdays 9AM-7PM
...,...,...,...,...,...,...,...,...,...
3339,6498,Am Bismarckplatz,"Schneidmühlstraße 5, 69115 Heidelberg",49.409666,8.691907,1.6,42.0,21.0,Monday-Sunday 9AM-11PM
3340,10037,Parkhaus Benediktinerplatz,"Benediktinerplatz 8, 78467 Konstanz",47.668773,9.177277,1.6,118.0,51.0,Weekdays 9AM-11PM
3341,17426,Parkplatz,Eglosheim,48.909889,9.181621,3.8,140.0,NaN,Weekdays 6AM-6PM
3342,17439,Parkplatz,Marbach (Neckar),48.943490,9.262973,3.8,67.0,NaN,Weekdays 6AM-6PM


Data Cleaning

In [29]:
# # Clean the Freiburg parking data
df_freiburg_cleaned = df_freiburg.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"])  

# Clean the public parking data
df_public_parking_cleaned = df_public_parking.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"]) 

# Remove duplicates based on parking ID
df_freiburg_cleaned = df_freiburg_cleaned.drop_duplicates(subset=["public_id"])
df_public_parking_cleaned = df_public_parking_cleaned.drop_duplicates(subset=["public_id"])

df_public_parking_cleaned


,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",48.495983,9.210331,3.8,198.0,NaN,Weekdays 9AM-11PM
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",48.488955,9.216701,1.6,181.0,NaN,Weekdays 9AM-7PM
2,17151,Parkplatz,Stuttgart,48.834065,9.152340,3.8,200.0,NaN,Weekdays 9AM-11PM
3,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",48.497575,9.211077,1.6,128.0,NaN,Weekdays 9AM-7PM
5,759,Tannenberger Straße,"Tannenberger Straße, Reutlingen",48.511669,9.204728,3.8,46.0,NaN,Monday-Sunday 9AM-11PM
...,...,...,...,...,...,...,...,...,...
3339,6498,Am Bismarckplatz,"Schneidmühlstraße 5, 69115 Heidelberg",49.409666,8.691907,1.6,42.0,21.0,Monday-Sunday 9AM-11PM
3340,10037,Parkhaus Benediktinerplatz,"Benediktinerplatz 8, 78467 Konstanz",47.668773,9.177277,1.6,118.0,51.0,Weekdays 9AM-11PM
3341,17426,Parkplatz,Eglosheim,48.909889,9.181621,3.8,140.0,NaN,Weekdays 6AM-6PM
3342,17439,Parkplatz,Marbach (Neckar),48.943490,9.262973,3.8,67.0,NaN,Weekdays 6AM-6PM


Data Integration

In [30]:
# Ensure latitude and longitude columns are numeric and consistent
df_freiburg_cleaned["latitude"] = pd.to_numeric(df_freiburg_cleaned["latitude"], errors="coerce")
df_freiburg_cleaned["longitude"] = pd.to_numeric(df_freiburg_cleaned["longitude"], errors="coerce")

df_public_parking_cleaned["latitude"] = pd.to_numeric(df_public_parking_cleaned["latitude"], errors="coerce")
df_public_parking_cleaned["longitude"] = pd.to_numeric(df_public_parking_cleaned["longitude"], errors="coerce")

# Drop rows with NaN in latitude or longitude
df_freiburg_cleaned = df_freiburg_cleaned.dropna(subset=["latitude", "longitude"])
df_public_parking_cleaned = df_public_parking_cleaned.dropna(subset=["latitude", "longitude"])

# Round latitude and longitude to the same precision for matching
df_freiburg_cleaned["latitude"] = df_freiburg_cleaned["latitude"].round(6)
df_freiburg_cleaned["longitude"] = df_freiburg_cleaned["longitude"].round(6)

df_public_parking_cleaned["latitude"] = df_public_parking_cleaned["latitude"].round(6)
df_public_parking_cleaned["longitude"] = df_public_parking_cleaned["longitude"].round(6)

# Merge the DataFrames, prioritizing df_public_parking_cleaned data
df_combined = pd.merge(
    df_freiburg_cleaned,
    df_public_parking_cleaned,
    on=["latitude", "longitude"],
    how="outer",
    suffixes=("_freiburg", "_public")
)

# Prioritize data from df_public_parking_cleaned
for column in df_public_parking_cleaned.columns:
    if column not in ["latitude", "longitude"]:  # Exclude keys used for merging
        df_combined[column] = df_combined[column + "_public"].combine_first(df_combined[column + "_freiburg"])

# Drop redundant columns
columns_to_drop = [col for col in df_combined.columns if col.endswith("_freiburg") or col.endswith("_public")]
df_combined = df_combined.drop(columns=columns_to_drop)

df_combined



,latitude,longitude,public_id,name,address,price_per_hour,capacity,available_space,opening_time
0,47.520513,8.585892,9352.0,Embrach-Rorbas,"Bahnstrasse 9, 8424 Embrach-Rorbas",3.2,28.0,NaN,Weekdays 9AM-7PM
1,47.522619,7.689327,9354.0,Pratteln,"Güterstrasse 19, 4133 Pratteln",1.6,58.0,NaN,Weekdays 9AM-11PM
2,47.523924,8.537127,19807.0,Bülach,"Schaffhauserstrasse 100, 8180 Bülach",3.2,234.0,NaN,Weekdays 6AM-6PM
3,47.524211,9.202017,8993.0,Kradolf,"Hauptstrasse 57, 9214 Kradolf",3.8,16.0,NaN,Weekdays 9AM-7PM
4,47.525776,8.775885,9116.0,Wiesendangen,"Alte Frauenfelderstrasse 3, 8542 Wiesendangen",1.6,35.0,NaN,Weekdays 9AM-11PM
...,...,...,...,...,...,...,...,...,...
2531,49.762093,9.512663,17773.0,Parkplatz,Wertheim (TBB),3.8,9.0,NaN,Monday-Sunday 9AM-11PM
2532,49.762799,9.512175,17771.0,Parkplatz,Wertheim (TBB),3.8,3.0,NaN,Weekdays 9AM-11PM
2533,49.785998,9.501370,17777.0,Parkplatz,Bestenheid,3.2,14.0,NaN,Monday-Sunday 9AM-11PM
2534,49.786031,9.500884,17776.0,Parkplatz,Bestenheid,1.6,11.0,NaN,Weekdays 6AM-6PM


In [31]:
# Define desired column order
desired_columns = [
    "public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity", "available_space", "opening_time"
]

# Reorder the columns in the combined DataFrame
df_combined = df_combined[[col for col in desired_columns if col in df_combined.columns]]

df_combined['public_id'] = df_combined['public_id'].apply(lambda x: f"PUB{random.randint(10000, 99999)}")

df_combined

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time
0,PUB77210,Embrach-Rorbas,"Bahnstrasse 9, 8424 Embrach-Rorbas",47.520513,8.585892,3.2,28.0,NaN,Weekdays 9AM-7PM
1,PUB66187,Pratteln,"Güterstrasse 19, 4133 Pratteln",47.522619,7.689327,1.6,58.0,NaN,Weekdays 9AM-11PM
2,PUB66177,Bülach,"Schaffhauserstrasse 100, 8180 Bülach",47.523924,8.537127,3.2,234.0,NaN,Weekdays 6AM-6PM
3,PUB41417,Kradolf,"Hauptstrasse 57, 9214 Kradolf",47.524211,9.202017,3.8,16.0,NaN,Weekdays 9AM-7PM
4,PUB81064,Wiesendangen,"Alte Frauenfelderstrasse 3, 8542 Wiesendangen",47.525776,8.775885,1.6,35.0,NaN,Weekdays 9AM-11PM
...,...,...,...,...,...,...,...,...,...
2531,PUB53268,Parkplatz,Wertheim (TBB),49.762093,9.512663,3.8,9.0,NaN,Monday-Sunday 9AM-11PM
2532,PUB88193,Parkplatz,Wertheim (TBB),49.762799,9.512175,3.8,3.0,NaN,Weekdays 9AM-11PM
2533,PUB89462,Parkplatz,Bestenheid,49.785998,9.501370,3.2,14.0,NaN,Monday-Sunday 9AM-11PM
2534,PUB41737,Parkplatz,Bestenheid,49.786031,9.500884,1.6,11.0,NaN,Weekdays 6AM-6PM


In [32]:
df_combined.to_json('public_parking.json', orient='records', indent=4)

print("DataFrame has been saved to 'public_parking.json'")

DataFrame has been saved to 'public_parking.json'
